In [1]:
from IPython.display import HTML
from IPython.display import display, Image
import casadi as ca
from casadi import jacobian
from casadi import gradient
from casadi import Function
from casadi import vertcat

<h1 align="center" style="font-size: 35px;"><font color="C1772D"><strong>Aplicación del Control Óptimo en el Campo de la Robótica</strong></font></h1>

<h2 align="center"><font color="#667388"><strong>Luis F. Recalde</strong><br>Ambato-Ecuador</font></h2>


<h3 align="center"><font color="#667388"><strong>Universidad Indoamerica</strong></font></h3>

<h3 style="font-size: 30px; color: #C1772D; font-weight: bold; text-align: left;">Casadi</h3>


<h3 align="left" style="font-size: 20px; color: #667388;"><strong>Introducción</strong></h3>


<div style="text-align: justify; font-size: 20px;">CasADi es una herramienta de software de código abierto para optimización numérica en general y control óptimo.</div>


<div style="text-align: justify; font-size: 20px;"> Se va formular y manipular expresiones en el marco simbólico de CasADi, generar información sobre derivadas de manera eficiente utilizando, establecer, resolver y realizar análisis de sensibilidad directo de ecuaciones diferenciales ordinarias (ODE) o ecuaciones diferenciales-algebraicas (DAE), así como formular y resolver problemas de programación no lineal (NLP) y problemas de control óptimo (OCP).</div>


<div style="text-align: justify; font-size: 20px;"> CasADi comenzó como una herramienta para la diferenciación algorítmica (AD) utilizando una sintaxis tomada de los sistemas de álgebra computacional (CAS), lo que explica su nombre. Aunque la AD sigue siendo una de las funcionalidades principales de la herramienta, el alcance de esta se ha ampliado considerablemente, con la adición de soporte para la integración y análisis de sensibilidad de ODE/DAE, programación no lineal e interfaces con otras herramientas numéricas. En su forma actual, es una herramienta de propósito general para la optimización numérica basada en gradientes, con un enfoque fuerte en el control óptimo.</div>


<div style="text-align: justify; font-size: 20px;"> CasADi no es un “solucionador de problemas de control óptimo” que permita al usuario introducir un OCP y luego devolver la solución. En cambio, intenta proporcionar al usuario un conjunto de “bloques de construcción” que pueden utilizarse para implementar solucionadores de OCP de propósito general o específico de manera eficiente con un esfuerzo de programación moderado.</div>


In [2]:
# Path to your image file
image_path = 'casadi.png'

# Specify the width for the image (adjust the value as needed)
image_width = 400

# HTML code for centering the image
html_code = f'<div style="text-align:center;"><img src="{image_path}" width="{image_width}"></div>'

# Display the HTML code
display(HTML(html_code))

<h3 align="left" style="font-size: 20px; color: #667388;"><strong>Simbologia SX</strong></h3>


<div style="text-align: justify; font-size: 20px;"> El tipo de dato SX se utiliza para representar matrices cuyos elementos consisten en expresiones simbólicas formadas por una secuencia de operaciones unarias y binarias.</div>

<div class="alert alert-block alert-success" style="font-size: 16px;">  
  <b>Code:</b> 
</div>

In [3]:
from casadi import *

In [4]:
x = SX.sym("x")

<div style="text-align: justify; font-size: 20px;"> Con esto se creara una matriz de $\mathbb{R}^{1 \times 1}$, la cual contiene la variable simbolica $x$. </div>

In [1]:
x

NameError: name 'x' is not defined

<div style="text-align: justify; font-size: 20px;">De igual forma se puede crear matrices y vector de variables simbolicas, incluyendo argumentos adicionales a SX.sym()</div>

In [6]:
y = SX.sym('y', 5)
Z = SX.sym('z', 4, 2)

In [7]:
y.shape

(5, 1)

In [8]:
Z.shape

(4, 2)

In [9]:
y

SX([y_0, y_1, y_2, y_3, y_4])

In [10]:
Z

SX(
[[z_0, z_4], 
 [z_1, z_5], 
 [z_2, z_6], 
 [z_3, z_7]])

<div style="text-align: justify; font-size: 20px;">De esta manera se crean las variables simbólicas de la siguiente forma: $\mathbf{y} \in \mathbb{R}^{5}$ y $\mathbf{Z} \in \mathbb{R}^{4 \times 2}$.</div>

<div style="text-align: justify; font-size: 20px;">Una vez creadas las variables, es posible formar expresiones de manera muy intuitiva.</div>

In [11]:
f = x**2 + 10
print(f)

(sq(x)+10)


In [12]:
d = sqrt(f)
print(d)

sqrt((sq(x)+10))


<div style="text-align: justify; font-size: 20px;">De igual forma, se pueden crear instancias de SX sin especificar valores simbólicos.</div>

In [13]:
B1 = SX.zeros(4, 5)

In [14]:
B1

SX(@1=0, 
[[@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1]])

<div style="text-align: justify; font-size: 20px;">$\mathbf{B}_1 \in \mathbb{R}^{4 \times 5}$ es una matriz densa vacía cuyos elementos son ceros, realmente son ceros.</div>

In [15]:
B2 = SX(4, 5)

In [16]:
B2

SX(
[[00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00]])

<div style="text-align: justify; font-size: 20px;">$\mathbf{B}_2 \in \mathbb{R}^{4 \times 5}$ es una matriz dispersa vacía cuyos elementos son ceros, ceros estructurales.</div>

In [17]:
B3 = SX.eye(4)

In [18]:
B3

SX(@1=1, 
[[@1, 00, 00, 00], 
 [00, @1, 00, 00], 
 [00, 00, @1, 00], 
 [00, 00, 00, @1]])

<div style="text-align: justify; font-size: 20px;">$\mathbf{B}_3 \in \mathbb{R}^{4 \times 4}$ es una matriz dispersa con unos en la diagonal</div>

<div style="text-align: justify; font-size: 20px;">Los ceros estructurales no se computan, a diferencia de los ceros reales, que sí son computados a pesar de que su resultado sea cero.</div>

<h3 align="left" style="font-size: 20px; color: #667388;"><strong>Simbologia MX</strong></h3>


<div style="text-align: justify; font-size: 20px;">Construir las siguientes operaciones usando SX, lo que resulta en:</div>

In [19]:
x = SX.sym('x', 2, 2)

In [20]:
print(x)


[[x_0, x_2], 
 [x_1, x_3]]


In [21]:
y = SX.sym('y')

In [22]:
print(y)

y


In [23]:
f = 3*x + y

In [24]:
print(f)

@1=3, 
[[((@1*x_0)+y), ((@1*x_2)+y)], 
 [((@1*x_1)+y), ((@1*x_3)+y)]]


In [25]:
f.shape

(2, 2)

<div style="text-align: justify; font-size: 20px;">Obsérvese cómo las expresiones son computadas elemento por elemento.</div>

<div style="text-align: justify; font-size: 20px;">La función MX, al igual que SX, permite formar expresiones que consisten en operaciones elementales. Sin embargo, a diferencia de SX, en MX estas pueden ser más generales, permitiendo realizar operaciones directamente entre matrices y vectores.</div>

In [26]:
x = MX.sym('x', 2, 2)

In [27]:
print(x)

x


In [28]:
y = MX.sym('y')

In [29]:
print(y)

y


In [30]:
f = 3*x + y

In [31]:
print(f)

((3*x)+y)


<div style="text-align: justify; font-size: 20px;">Obsérvese cómo el resultado usando MX solo consta de dos operaciones, mientras que con SX se obtiene un total de 8 operaciones. Como consecuencia, MX puede ser más económica de computar cuando se trabaja con operaciones que son parte natural de vectores y matrices.</div>

<div style="text-align: justify; font-size: 20px;">Para acceder a la información dentro de los elementos creados se tiene lo siguiente:</div>

In [32]:
x = MX.sym('x', 2, 2)

In [33]:
print(x)

x


In [34]:
x.shape

(2, 2)

In [35]:
print(x[0, 0])

x[0]


<div style="text-align: justify; font-size: 20px;">Para modificar la información dentro de las matrices creadas se realiza lo siguiente:</div>

In [36]:
x = MX.sym('x', 2)

In [37]:
x

MX(x)

In [38]:
A = MX(2, 2)

In [39]:
A

MX(zeros(2x2,0nz))

In [40]:
A[0, 0] = x[0, 0]

In [41]:
A[1, 1] = x[0, 0] + x[1, 0]

In [42]:
print(A)

(project((zeros(2x2,1nz)[0] = x[0]))[1] = (x[0]+x[1]))


<h3 align="left" style="font-size: 20px; color: #667388;"><strong> Crear Matrices, acceder y modifcar valores</strong></h3>


<div style="text-align: justify; font-size: 20px;"> Para asignar un elemento o varios dentro de las matrices usando CasADi (SX, MX, DM), se utilizan corchetes y se inicia desde el cero.</div>

In [43]:
M = SX([[1, 2], [3, 6]])

In [44]:
M

SX(
[[1, 2], 
 [3, 6]])

In [45]:
print(M[0, :])

[[1, 2]]


In [46]:
print(M[:, 0])

[1, 3]


In [47]:
M[:, 0] = 1.0

In [48]:
print(M)

@1=1, 
[[@1, 2], 
 [@1, 6]]


<h3 align="left" style="font-size: 20px; color: #667388;"><strong> Operaciones aritmeticas</strong></h3>


<div style="text-align: justify; font-size: 20px;">Casadi soporta muchas de las operaciones aritméticas básicas, como la suma, multiplicación y funciones trigonométricas.</div>

In [49]:
x = SX.sym('x')

In [50]:
y = SX.sym('y', 2, 2)

In [51]:
f = sin(y) - x

In [52]:
print(f)


[[(sin(y_0)-x), (sin(y_2)-x)], 
 [(sin(y_1)-x), (sin(y_3)-x)]]


In [53]:
h = y*y

In [54]:
h

SX(
[[sq(y_0), sq(y_2)], 
 [sq(y_1), sq(y_3)]])

In [55]:
z = y@y

In [56]:
z

SX(
[[(sq(y_0)+(y_2*y_1)), ((y_0*y_2)+(y_2*y_3))], 
 [((y_1*y_0)+(y_3*y_1)), ((y_1*y_2)+sq(y_3))]])

In [57]:
x = MX.sym('x')

In [58]:
y = MX.sym('y', 2, 2)

In [59]:
f = sin(y) - x

In [60]:
print(f)

(sin(y)-x)


In [61]:
h = y*y

In [62]:
h

MX(sq(y))

In [63]:
z = y@y

In [64]:
z

MX(mac(y,y,zeros(2x2)))

In [65]:
y = SX.sym('y', 2, 2)

In [66]:
y

SX(
[[y_0, y_2], 
 [y_1, y_3]])

In [67]:
z = y.T

In [68]:
z

SX(
[[y_0, y_1], 
 [y_2, y_3]])

In [69]:
y = MX.sym('y', 2, 2)

In [70]:
y

MX(y)

In [71]:
z = y.T

In [72]:
z

MX(y')

<h3 align="left" style="font-size: 20px; color: #667388;"><strong> Redimensionar y concatenar variables</strong></h3>

In [73]:
x = SX.sym('x', 5)

In [74]:
y = SX.sym('y', 5)

In [75]:
z = vertcat(x, y)

In [76]:
z

SX([x_0, x_1, x_2, x_3, x_4, y_0, y_1, y_2, y_3, y_4])

z.shape

In [77]:
h = horzcat(x, y)

In [78]:
h

SX(
[[x_0, y_0], 
 [x_1, y_1], 
 [x_2, y_2], 
 [x_3, y_3], 
 [x_4, y_4]])

In [79]:
h.shape

(5, 2)

<h3 align="left" style="font-size: 20px; color: #667388;"><strong> Algebra Lineal </strong></h3>

<div style="text-align: justify; font-size: 20px;">Casadi soporta un cierto número de operaciones de álgebra lineal para la solución de sistemas de ecuaciones lineales.</div>

In [80]:
A = MX.sym('A', 3, 3)

In [81]:
b = MX.sym('b', 3, 1)

In [82]:
x = solve(A, b)

In [83]:
x

MX((A\b))

In [84]:
solver = Function('solver', [A, b], [x])

In [85]:
A_val = np.array([[9, 3, 1], [1, 1, 4], [0, 5, 2]])
b_val = np.array([10, 15, 20])


In [86]:
x_val = solver(A_val, b_val)

In [87]:
x_val

DM([-0.153374, 2.76074, 3.09816])

<div style="text-align: justify; font-size: 20px;">Usando SX o DM</div>

In [88]:
A = DM([[9, 3, 1], [1, 1, 4], [0, 5, 2]])

In [89]:
b = DM([10, 15, 20])

In [90]:
x = solve(A, b)

In [91]:
x

DM([-0.153374, 2.76074, 3.09816])

<div style="text-align: justify; font-size: 20px;">MX con valores numericos?</div>

In [92]:
A_dm = DM([[9, 3, 1],
              [1, 1, 4],
              [0, 5, 2]])

# Convert DM to MX
A = ca.MX(A_dm)


print("Matrix A:")
print(A)

Matrix A:

[[9, 3, 1], 
 [1, 1, 4], 
 [0, 5, 2]]


In [93]:
b = MX([10, 15, 20])

In [94]:
b

MX([10, 15, 20])

In [95]:
x =  solve(A, b)

In [96]:
x

MX((
[[9, 3, 1], 
 [1, 1, 4], 
 [0, 5, 2]]\[10, 15, 20]))

In [97]:
solucion = Function('solucion', [A, b], [x])

RuntimeError: Error in Function::Function for 'solucion' [MXFunction] at .../casadi/core/function.cpp:235:
.../casadi/core/function_internal.cpp:144: Error calling MXFunction::init for 'solucion':
.../casadi/core/x_function.hpp:284: Xfunction input arguments must be purely symbolic. 
Argument 0(i0) is not symbolic.


<h3 align="left" style="font-size: 20px; color: #667388;"><strong> Diferenciacion en Casadi </strong></h3>

In [98]:
A = SX.sym('A',3,2)

In [99]:
A

SX(
[[A_0, A_3], 
 [A_1, A_4], 
 [A_2, A_5]])

In [100]:
x = SX.sym('x',2)

In [101]:
x

SX([x_0, x_1])

In [102]:
xp = A@x

In [103]:
xp

SX([((A_0*x_0)+(A_3*x_1)), ((A_1*x_0)+(A_4*x_1)), ((A_2*x_0)+(A_5*x_1))])

In [104]:
J = jacobian(xp, x)

In [105]:
J

SX(
[[A_0, A_3], 
 [A_1, A_4], 
 [A_2, A_5]])

In [106]:
interno = dot(x, x)

In [107]:
interno

SX((sq(x_0)+sq(x_1)))

In [108]:
interno.shape

(1, 1)

In [109]:
J = jacobian(interno, x)

In [110]:
J

SX([[(x_0+x_0), (x_1+x_1)]])

In [111]:
J.shape

(1, 2)

In [112]:
gradiente  = gradient(interno, x)

In [113]:
gradiente

SX([(x_0+x_0), (x_1+x_1)])

In [114]:
gradiente.shape

(2, 1)

<h3 align="left" style="font-size: 20px; color: #667388;"><strong> Funciones en CasADi </strong></h3>

In [115]:
x = SX.sym('x',2)
y = SX.sym('y')
f = Function('f',[x,y], [x,sin(y)*x])
print(f)

f:(i0[2],i1)->(o0[2],o1[2]) SXFunction


In [116]:
x = MX.sym('x',2)
y = MX.sym('y')
f = Function('f',[x,y],[x,sin(y)*x])
print(f)

f:(i0[2],i1)->(o0[2],o1[2]) MXFunction


In [117]:
f = Function('f',[x,y],[x,sin(y)*x],['x','y'],['r','q'])

In [118]:
f

Function(f:(x[2],y)->(r[2],q[2]) MXFunction)

In [119]:
res = f(x = [1.1, 2], y = [3.3])

In [120]:
res

{'q': DM([-0.17352, -0.315491]), 'r': DM([1.1, 2])}

In [121]:
x =  SX.sym('x',3)

In [139]:
Q = DM.eye(3)

In [147]:
value  = Q@x

In [148]:
dv_dx_f = jacobian(value, x)

In [149]:
dv_dx_f

SX(@1=1, 
[[@1, 00, 00], 
 [00, @1, 00], 
 [00, 00, @1]])

In [150]:
f =  Function('f',[x],[value],['x'],['value'])

In [151]:
dv_dx = Function('dv_dx',[x],[dv_dx_f],['x'],['dv_dx'])

<div style="text-align: justify; font-size: 20px;">Verificar Funciones</div>

In [152]:
valor = f([1, 2, 3])
valor

DM([1, 2, 3])

In [153]:
derivada = dv_dx([1,2,5])

In [155]:
derivada[:, :]

DM(
[[1, 00, 00], 
 [00, 1, 00], 
 [00, 00, 1]])

In [156]:
aux = np.array(derivada)

In [157]:
aux

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])